In [1]:
# install libraries
!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import imblearn
print(imblearn.__version__)

0.12.0


In [3]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
# ingest data
data_2022 = "FY2022_068_Contracts_Full_20240214_1.csv"
data_2023 = "FY2023_068_Contracts_Full_20240214_1.csv"
data_2024 = "FY2024_068_Contracts_Full_20240214_1.csv"
df_2022 = pd.read_csv(data_2022)
df_2023 = pd.read_csv(data_2023)
df_2024 = pd.read_csv(data_2024)
df_combined = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

/tmp/ipykernel_2579308/479897428.py:5: DtypeWarning: Columns (47,68,71,81,164,165) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv(data_2022)
/tmp/ipykernel_2579308/479897428.py:6: DtypeWarning: Columns (47,52,68,81,164,165) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv(data_2023)


In [6]:
# retain desired fields
field_names = [    
    "potential_total_value_of_award",
    "recipient_state_code",
    "alaskan_native_corporation_owned_firm",
    "native_hawaiian_organization_owned_firm",
    "subcontinent_asian_asian_indian_american_owned_business",
    "asian_pacific_american_owned_business",
    "black_american_owned_business",
    "hispanic_american_owned_business",
    "native_american_owned_business",
    "other_minority_owned_business",
    "veteran_owned_business",
    "woman_owned_business",
    "contracting_officers_determination_of_business_size"
]
df_combined = df_combined[field_names]

In [7]:
# drop duplicates
print("Original shape:", df_combined.shape)
df_combined = df_combined.drop_duplicates()
print("Shape after removing duplicates:", df_combined.shape)

Original shape: (28939, 13)
Shape after removing duplicates: (13618, 13)


In [8]:
# remove outliers
Q1 = df_combined['potential_total_value_of_award'].quantile(0.25)
Q3 = df_combined['potential_total_value_of_award'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df_combined[(df_combined['potential_total_value_of_award'] >= lower_bound) & (df_combined['potential_total_value_of_award'] <= upper_bound)]

print("Original shape:", df_combined.shape)
print("Shape after removing outliers:", df_filtered.shape)

Original shape: (13618, 13)
Shape after removing outliers: (11679, 13)


In [9]:
# drop nulls and zeros
print("Before dropping nulls", df_filtered.shape)
df_filtered = df_filtered.dropna()
print("After dropping nulls", df_filtered.shape)

print("After dropping zeros", df_filtered.shape)
df_filtered = df_filtered[df_filtered['potential_total_value_of_award'] != 0]
print("After dropping zeros", df_filtered.shape)

Before dropping nulls (11679, 13)
After dropping nulls (11629, 13)
After dropping zeros (11629, 13)
After dropping zeros (11557, 13)


In [10]:
# rename columns for clarity
df_filtered.rename(columns={
    'recipient_state_code': 'region_code',
    'alaskan_native_corporation_owned_firm': 'alaskan_native_owned',
    'native_hawaiian_organization_owned_firm': 'hawaiian_native_owned',
    'subcontinent_asian_asian_indian_american_owned_business': 'asian_owned',
    'asian_pacific_american_owned_business': 'asian_pacific_owned',
    'black_american_owned_business': 'black_owned',
    'hispanic_american_owned_business': 'hispanic_owned',
    'native_american_owned_business': 'native_american_owned',
    'other_minority_owned_business': 'other_minority_owned',
    'contracting_officers_determination_of_business_size': 'business_size',
    'veteran_owned_business': 'veteran_owned',
    'woman_owned_business': 'women_owned'
}, inplace=True)

In [11]:
# replace t's and f's for 1's and 0's
df_filtered = df_filtered.replace({'t': 1, 'f': 0})

In [12]:
# create ethnicity_sum column
df_filtered['ethnicity_sum'] = df_filtered.alaskan_native_owned + df_filtered.hawaiian_native_owned + df_filtered.asian_owned + df_filtered.asian_pacific_owned + df_filtered.black_owned + df_filtered.hispanic_owned + df_filtered.native_american_owned + df_filtered.other_minority_owned
df_filtered = df_filtered[df_filtered['ethnicity_sum'] != 2]
df_filtered['ethnicity_sum'] = df_filtered['ethnicity_sum'].replace({0: 1})

In [13]:
# one hot encoding and column renaming
df_filtered = pd.get_dummies(df_filtered, columns=['business_size'])
df_filtered.drop('business_size_OTHER THAN SMALL BUSINESS', axis=1, inplace=True)
df_filtered.rename(columns={'ethnicity_sum': 'white_owned'}, inplace=True)

In [14]:
# map original ethnicity value to new ethnicity column
for index, row in df_filtered.iterrows():
    if row['alaskan_native_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'alaskan_native_owned'
    elif row['hawaiian_native_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'hawaiian_native_owned'
    elif row['asian_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'asian_owned'
    elif row['asian_pacific_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'asian_pacific_owned'
    elif row['black_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'black_owned'
    elif row['hispanic_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'hispanic_owned'
    elif row['native_american_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'native_american_owned'
    elif row['other_minority_owned'] == 1:
        df_filtered.at[index, 'ethnicity'] = 'other_minority_owned'
    else:
        df_filtered.at[index, 'ethnicity'] = 'white_owned'

In [15]:
# drop original ethnicity columns
drop = [
    'alaskan_native_owned',
    'hawaiian_native_owned',
    'asian_owned',
    'asian_pacific_owned',
    'black_owned',
    'hispanic_owned',
    'native_american_owned',
    'other_minority_owned',
    'white_owned'
]
df_filtered.drop(drop, axis=1, inplace=True)

In [16]:
# rename column for clarity
df_filtered.rename(columns={'business_size_SMALL BUSINESS': 'small_business'}, inplace=True)

In [17]:
# remove rows where ethnicity is white, retain rows where business is women-owned and drop women-owned column
print(df_filtered.shape)
df_filtered = df_filtered[df_filtered['ethnicity'] != 'white_owned']
df_filtered = df_filtered[df_filtered['women_owned'] == 1]
df_filtered.drop(['women_owned'], axis=1, inplace=True)
print(df_filtered.shape)

(11070, 6)
(415, 5)


In [21]:
df_filtered

,potential_total_value_of_award,region_code,veteran_owned,small_business,ethnicity,training_or_user_input
0,1127006.90,UT,0,1,native_american_owned,training
92,246994.00,OH,0,1,asian_owned,training
150,1566745.82,CO,0,1,asian_pacific_owned,training
163,500000.00,MD,0,1,black_owned,training
309,236221.04,MD,0,1,hispanic_owned,training
...,...,...,...,...,...,...
28733,79714.00,OH,0,1,asian_owned,training
28796,925720.00,OH,0,1,asian_owned,training
28857,1026316.99,MD,0,1,asian_owned,training
28893,638003.00,OH,0,1,asian_owned,training


# here is where you add the 'output row'

In [22]:
df_filtered['training_or_user_input'] = 'training'
potential_award_median = df_filtered['potential_total_value_of_award'].median()
user_input = [potential_award_median, 'CA', 1, 1, 'asian_owned', 'user_input']
df_filtered = df_filtered.append(pd.Series(user_input, index=df_filtered.columns), ignore_index=True)
df_filtered

/tmp/ipykernel_2579308/1281141437.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_filtered = df_filtered.append(pd.Series(user_input, index=df_filtered.columns), ignore_index=True)


,potential_total_value_of_award,region_code,veteran_owned,small_business,ethnicity,training_or_user_input
0,1127006.90,UT,0,1,native_american_owned,training
1,246994.00,OH,0,1,asian_owned,training
2,1566745.82,CO,0,1,asian_pacific_owned,training
3,500000.00,MD,0,1,black_owned,training
4,236221.04,MD,0,1,hispanic_owned,training
...,...,...,...,...,...,...
411,925720.00,OH,0,1,asian_owned,training
412,1026316.99,MD,0,1,asian_owned,training
413,638003.00,OH,0,1,asian_owned,training
414,688195.00,OH,0,1,asian_owned,training


In [23]:
# map state code to larger-encompassing region already used by EPA
region_mapping = {
    'CT': 1, 'ME': 1, 'MA': 1, 'NH': 1, 'RI': 1, 'VT': 1,
    'NJ': 2, 'NY': 2, 'PR': 2,
    'DE': 3, 'DC': 3, 'MD': 3, 'PA': 3, 'VA': 3, 'WV': 3,
    'AL': 4, 'FL': 4, 'GA': 4, 'KY': 4, 'MS': 4, 'NC': 4, 'SC': 4, 'TN': 4,
    'IL': 5, 'IN': 5, 'MI': 5, 'MN': 5, 'OH': 5, 'WI': 5,
    'AR': 6, 'LA': 6, 'NM': 6, 'OK': 6, 'TX': 6,
    'IA': 7, 'KS': 7, 'MO': 7, 'NE': 7,
    'CO': 8, 'MT': 8, 'ND': 8, 'SD': 8, 'UT': 8, 'WY': 8,
    'AZ': 9, 'CA': 9, 'HI': 9, 'NV': 9,
    'AK': 10, 'ID': 10, 'OR': 10, 'WA': 10
}
df_filtered['region_code'] = df_filtered['region_code'].map(region_mapping)
df_filtered.dropna(inplace=True)
df_filtered.loc[:, 'region_code'] = df_filtered['region_code'].astype(int)

In [24]:
# label encode ethnicity column
label_encoder = LabelEncoder()
df_filtered['ethnicity'] = label_encoder.fit_transform(df_filtered['ethnicity'])

In [25]:
# target-mean and target-median encoding
target_median_encoding_region_code = df_filtered.groupby('region_code')['potential_total_value_of_award'].median()
target_mean_encoding_veteran_owned = df_filtered.groupby('veteran_owned')['potential_total_value_of_award'].mean()
#target_mean_encoding_women_owned = df_filtered.groupby('women_owned')['potential_total_value_of_award'].mean()
target_mean_encoding_small_business = df_filtered.groupby('small_business')['potential_total_value_of_award'].mean()
target_median_encoding_ethnicity = df_filtered.groupby('ethnicity')['potential_total_value_of_award'].median()

df_filtered['region_code'] = df_filtered['region_code'].map(target_median_encoding_region_code)
df_filtered['veteran_owned'] = df_filtered['veteran_owned'].map(target_mean_encoding_veteran_owned)
#df_filtered['women_owned'] = df_filtered['women_owned'].map(target_mean_encoding_women_owned)
df_filtered['small_business'] = df_filtered['small_business'].map(target_mean_encoding_small_business)
df_filtered['ethnicity'] = df_filtered['ethnicity'].map(target_median_encoding_ethnicity)

In [26]:
# standardization
fields_to_scale = ['region_code', 'small_business', 'veteran_owned', 'ethnicity']
scaler = StandardScaler()
df_filtered[fields_to_scale] = scaler.fit_transform(df_filtered[fields_to_scale])
df_filtered

,potential_total_value_of_award,region_code,veteran_owned,small_business,ethnicity,training_or_user_input
0,1127006.90,1.428980,-0.186617,0.186617,0.759886,training
1,246994.00,-0.338471,-0.186617,0.186617,-0.751234,training
2,1566745.82,1.428980,-0.186617,0.186617,-0.338040,training
3,500000.00,-0.464099,-0.186617,0.186617,0.356726,training
4,236221.04,-0.464099,-0.186617,0.186617,2.410234,training
...,...,...,...,...,...,...
411,925720.00,-0.338471,-0.186617,0.186617,-0.751234,training
412,1026316.99,-0.464099,-0.186617,0.186617,-0.751234,training
413,638003.00,-0.338471,-0.186617,0.186617,-0.751234,training
414,688195.00,-0.338471,-0.186617,0.186617,-0.751234,training


In [27]:
# bin target feature to discrete value based on tertiles for multiclass model
df_filtered_class = df_filtered.copy()
df_filtered_class['potential_total_value_of_award'] = pd.qcut(df_filtered['potential_total_value_of_award'], q=3, labels=[1, 2, 3])

In [33]:
# identifying tertiles
t1 = np.percentile(df_filtered['potential_total_value_of_award'], 33.33)
t2 = np.percentile(df_filtered['potential_total_value_of_award'], 66.66)
print("T1:", t1)
print("T2:", t2)

T1: 110270.75850000003
T2: 727153.1793900001


In [28]:
# isolate the user input from the dataframe
user_input = df_filtered_class[df_filtered_class['training_or_user_input'] == 'user_input'][['region_code', 'veteran_owned', 'small_business', 'ethnicity']]
df_filtered_class = df_filtered_class[df_filtered_class['training_or_user_input'] != 'user_input']
df_filtered_class = df_filtered_class[['potential_total_value_of_award', 'region_code', 'veteran_owned', 'small_business', 'ethnicity']]
user_input

In [29]:
user_input

,region_code,veteran_owned,small_business,ethnicity
415,-0.607393,5.358571,0.186617,-0.751234


In [30]:
# define dependent and independent variables in dataframe
X = df_filtered_class.drop(columns=['potential_total_value_of_award'])
y = df_filtered_class['potential_total_value_of_award']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# apply SMOTE to handle class imbalances
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [31]:
# Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_resampled, y_train_resampled)

# make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6144578313253012


In [32]:
y_pred = rf_classifier.predict(user_input)
y_pred

array([1])